# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# HW3

### import the necessary library

In [1]:
import json
import datetime
from collections import Counter


In [2]:
# For measuring the total running time of this notebook
start_notebook_t = datetime.datetime.now()
print(start_notebook_t)

2024-02-02 04:36:42.128482


In [3]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


24/02/02 04:36:43 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
24/02/02 04:36:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/02 04:36:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/02 04:36:44 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 8f:77:be:f6:47:f0:43:c7


### Read data

In [4]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [5]:
# tweets_rdd = tweets_rdd.sample(False, 0.1, seed=42)


### Show a sample of tweets

In [ ]:
tweets_rdd.takeSample(False, 1, 19)

['{"in_reply_to_status_id_str": null, "in_reply_to_user_id_str": null, "in_reply_to_status_id": null, "in_reply_to_user_id": null, "in_reply_to_screen_name": null, "truncated": false, "is_quote_status": false, "retweet_count": 3, "reply_count": 0, "quote_count": 0, "favorite_count": 0, "favorited": false, "retweeted": true, "possibly_sensitive": null, "lang": "fa", "geo": null, "view_count": 0, "tweet_type": "retweeted", "emojis": null, "text": "مردک وطن\u200cفروش بدبخت، ترانه آفتابکاران جنگل مال چی بوده که خوندن؟! موسیقی اصیل ایرانی بوده؟! یا سرود تروریست\u200cهای چریک کمونیست بوده؟!\\nاشکال نداره، شما به خزعبل گویی ادامه بدید. خوبیش اینه که مردم دارن کم کم متوجه میشن که شنا دگوری\u200cهای خائن وطن\u200cفروش چه جماعتی هستید.\\n\\n#ننگ_بر_سه_فاسد_ملا_چپی_مجاهد", "user": {"id": "315689091", "id_str": "315689091", "created_at": 1307868062, "name": "The Nerd", "screen_name": "NewbieBadASS", "location": "خیابون هفده شهریور ", "url": "https://t.co/ajpmxoYoTe", "description": "A supporter of

### Show size of data (No. tweets)

In [ ]:
tweets_rdd.count()

4175860

## Extracting the selected user's favorite tweet ids

### Define Parameters for the Primary Rating.

In [10]:
# Define Parameters
owner_s = 1

# following parameters should not be equal to owner_s
rep_to_s = 0.2
rep_from_s = 0.9

ret_to_s = 0.5
ret_from_s = 1.5

qut_to_s = 0.6
qut_from_s = 1.5


### Define functions for primary and secondary ratings

In [11]:
def to_user_tid_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for calculate primary rating matrix.
    
    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH USER.

    The output is a list of tuples, where each tuple contains
    a user ID and a list of tweets that have been rated (along
    with the value of the rating).
    Output format: [((u_i, ((t_j, s_ij),)) ...]
    '''
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(uid, [(tid, owner_s)])]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(uid, [(tid, owner_s), (origin_tid[0], from_s)]),
            (origin_uid[0], [(tid, to_s)]),
           ]


def to_tid_user_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for secondary rating.

    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH TWEET.

    The output is a list of tuples, where each tuple contains
    a tweet ID and a tuple list of users rate to it (along with
    the value of the rating).
    Output format: [(tid_i, ((u_j, s_ij), ...)), ...]
    '''
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(tid, ((uid, owner_s),))]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(tid, ((uid, owner_s), (origin_uid[0], to_s),)),
            (origin_tid[0], ((uid, from_s),)),
           ]


def extract_tuples(tweet_info, func):
    '''
    This function extract some information of tweet and send it
    to one of the following funcions and return their results:
    - to_user_tid_score
    - to_tid_user_score
    '''
    return func(
        tweet_info.get('id'),
        tweet_info["user"].get('id'),
        tweet_info.get('tweet_type'),
        [
            tweet_info.get('in_reply_to_status_id_str', None),
            tweet_info.get('quoted_status', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('id'),
        ],
        [
            tweet_info.get('in_reply_to_user_id_str', None),
            tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
        ],
    )


def extract_relations(x):
    '''
    This function extracts the MAX_REALTION strongest relationships
    between users who interacted with this tweet.
    
    x contains the tweet_id (tid) and a list of tuples. Each tuple
    contains a user_id (u) and its interaction with the tweet (score
    given to the tweet).
    x: (tid, [(u1, s1), ...])

    The output is a list that each element of it, contains the user_id
    (u1) and a list of tuples. Each tuple contains a tweet_id (tid), 
    second user_id that has relation to u1 (called u2), and the score
    given by each of these two users to this tweet (s1 and s2 respectively)
    output: 
    [(u1, [(tid, u2, s1, s2),
            (tid, u3, s1, s3),
            ...
           ]),
     (u2, [(tid, u1, s2, s1),
            (tid, u3, s2, s3),
            ...
                  ]),
     ...
    ]
    
    '''
    MAX_REALTION = 10

    tid, user_score = x
    relations = []
    threashold = rep_to_s
    if len(user_score) > MAX_REALTION:
        threashold = ret_to_s

    for (u1, s1) in user_score:
        u1_relations = []
        for (u2, s2) in user_score:
            if u1 != u2 and s1 > threashold and s1 != owner_s:
                u1_relations.append((tid, u2, s1, s2))

        if len(u1_relations) > MAX_REALTION:
            relations.append(
                (u1, sorted(u1_relations, key=lambda x: -x[2]*x[3])[:10]))
        else:
            relations.append((u1, u1_relations))
        
    return relations


def select_best(x):
    '''
    This function select 25 strongest relations from relations 
    exist in x.
    x contains the user_id (u1) and a list of tuples. Each tuple
    contains a tweet_id (tid), second user_id that has relation 
    to u1 (called u2), and the score given by each of these two
    users to this tweet (s1 and s2 respectively)

    x: (u1, [(tid, u2, s1, s2),])
    '''
    u1, relations = x
    if relations:
        th = rep_to_s
        while len(relations) > 25:
            new_relations = [r for r in relations if r[2]*r[3] > th]
            if len(new_relations) < 15:
                break
            relations = new_relations
            th *= 1.1

    return (u1, relations)


def find_user_favorites(x, user_scores):
    ''' 
        user_scores: {ui: [(t_j, s_ij), ...]}
        x: (u1, ((tid, uj, s1, sj), ...))
    '''
    th = 0.05
    u, relations = x
    favorite = {}
    if relations:
        for (tid, uj, s1, s2) in relations:
            favorite[tid] = favorite.get(tid, 0) + s1
            for (t, s) in user_scores.get(uj, []):
                favorite[t] = favorite.get(t, 0) + s1*s2*s

    if favorite:
        max_favorite = max(favorite.values())
        th = 0.05 * max_favorite
        while len(favorite) > 100:
            favorite = {t:i for t,i in favorite.items() if i > th}
            th *= 1.1
        return (u, favorite)
    return (u,{})


### Calculating Primary rating Matrix

In [12]:
start_calculating_matrix = datetime.datetime.now()
print(start_calculating_matrix)


2024-02-02 04:36:45.268771


In [13]:
# [((u_i, ((t_j, s_ij),)), ...]
user_tid_score_rdd = tweets_rdd.flatMap(
    lambda x: extract_tuples(json.loads(x), func=to_user_tid_score))

# [(u_i, ((t_j, s_ij), ...)), ...]
user_tids_scores_rdd = user_tid_score_rdd.reduceByKey(
    lambda x, y: x if not x.extend(y) else x)

user_scores_list = user_tids_scores_rdd.collect()

user_scores_dict = dict(user_scores_list)


### Calculate and determine favirite tweets for each user


In [14]:
start_extracting_tweets_id_t = datetime.datetime.now()
print(start_extracting_tweets_id_t)

2024-02-02 04:37:22.042981


In [15]:
# [(tid, [(u, s)]), ...]
tid_user_score_rdd = tweets_rdd.flatMap(lambda x: extract_tuples(json.loads(x), func=to_tid_user_score))

# [(tid, [(u, s), ...]), ...]
tid_users_scores_rdd = tid_user_score_rdd.reduceByKey(lambda x, y: x + y)

# [(u1, [(tid, u2, s1, s2)])]
user_relation_rdd = tid_users_scores_rdd.flatMap(extract_relations)

# [(u1, [(tid, u2, s1, s2), ...])]
user_relations_rdd = user_relation_rdd.reduceByKey(lambda x, y: x if not x.extend(y) else x)

# [(u1, [(tid, u2, s1, s2), ...])]
user_best_relations_rdd = user_relations_rdd.map(select_best)

# [(u1, [(tid: favorite), ...])]
user_favorites_rdd = user_best_relations_rdd.map(lambda x: find_user_favorites(x, user_scores_dict))


### select a user and stract its favorite tweet IDs

In [16]:
q_uid = list(user_scores_dict.keys())[1]
print('query user id:', q_uid)
user_favorites = user_favorites_rdd.filter(lambda x: x[0] == q_uid).take(1)

query user id: 1425068264862978049


In [17]:
user_favorites

[('1425068264862978049',
  {'1735288018141905193': 266.625,
   '1734602278693392524': 194.775,
   '1735571814368088148': 265.95000000000005,
   '1734489680467423678': 167.40000000000012,
   '1735584086117482503': 209.25,
   '1735828679127625919': 168.75000000000003,
   '1736364371687735299': 177.75,
   '1734665832343425153': 589.875,
   '1735290953148117479': 182.925,
   '1736042418024149425': 337.5,
   '1735044430485213259': 272.925,
   '1734948591016304642': 177.525,
   '1736849290943778837': 162.225,
   '1735050433389908314': 337.65,
   '1735059420244222420': 206.55,
   '1735362716150165854': 306.75,
   '1735779679095689417': 218.70000000000005,
   '1735036267442921821': 202.5,
   '1735035401750855875': 219.375,
   '1735393338784976977': 296.325,
   '1734091157699952951': 280.125,
   '1734612642520617354': 202.5,
   '1734876386098102428': 218.025,
   '1736482819952750984': 214.65,
   '1734986742745190648': 164.025,
   '1735788788742316376': 384.89999999999986,
   '173498686420806459

In [18]:
if not user_favorites[0][1]:
    print('Please choose another User ID, this user has no favorite tweet in this database')

### Extracte and Show selected user's tweets and the his/her favorite tweets information

In [19]:
start_extract_and_show_tweets_t = datetime.datetime.now()
print(start_extract_and_show_tweets_t)


2024-02-02 04:38:41.003853


In [20]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                  'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                  'quoted_status', 'retweeted_status','profile_image_url',
                 ]


In [21]:
favorites_tweets = tweets_rdd.filter(lambda x: json.loads(x)['id'] in user_favorites[0][1].keys() or
                                               json.loads(x)['user']['id'] == q_uid)\
                             .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                             .collect()
favorites_tweets2 = favorites_tweets.copy()


### Define functions needed to separate tweets based on tweet_type and display tweets appropriately

In [22]:
def extract_user_origin_tweets(tweets, uid, rem=True):
    origin_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'generated':
            if t.get('user',{}).get('id','0') == uid:
                origin_tweets.append(t)
    if rem:
        for t in origin_tweets:
            tweets.remove(t)
    return origin_tweets


def extract_user_replied_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('user',{}).get('id','0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_user_quoted_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('user',{}).get('id','0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_user_retweeted_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('user',{}).get('id','0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_replied_to_user_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('in_reply_to_user_id_str', '0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_quoted_user_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('quoted_status', {}).get('user', {}).get('id', '0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_retweeted_user_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('retweeted_status', {}).get('user', {}).get('id', '0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_explore_tweets(tweets):
    return tweets


def show_tweets(tweets, tweets_type):
    print('-'*100)
    print(f'Number of {tweets_type}:', len(tweets), '\n')
    ts = [t['text'].replace('\u200c', '').split('\n') for t in tweets]
    for t in ts:
        print('-'*100)
        for line in t:
            print(line)
    print('-'*100)


### Separate tweets based on tweet_type and display them

In [23]:
uid = q_uid
favorites_tweets = favorites_tweets2.copy()
user_origin_tweets = extract_user_origin_tweets(favorites_tweets, uid)
user_quoted_tweets  = extract_user_quoted_tweets(favorites_tweets, uid)
user_retweeted_tweets  = extract_user_retweeted_tweets(favorites_tweets, uid)
user_replied_tweets  = extract_user_replied_tweets(favorites_tweets, uid)
replied_to_user_tweets  = extract_replied_to_user_tweets(favorites_tweets, uid)
quoted_user_tweets  = extract_quoted_user_tweets(favorites_tweets, uid)
retweeted_user_tweets  = extract_retweeted_user_tweets(favorites_tweets, uid)
explore_tweets = extract_explore_tweets(favorites_tweets)


In [24]:
show_tweets(user_origin_tweets, 'user_origin_tweets')

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 4 

----------------------------------------------------------------------------------------------------
ای مگس عرصه سیمرغ نه جولانگه توست
عرض خود میبری و زحمت ما میداری
تو به تقصیر خود افتادی از این در محروم
از که مینالی و فریاد چرا میداری

#رضا_عقده_ایی 
#حزب_کارتنی https://t.co/MpmdadysLc
----------------------------------------------------------------------------------------------------
نمایشگاه «آنچه باقی ماند»

 #فواد_مصطفی_سلطانی 
#اعدام_های_فرودگاه_سنندج https://t.co/MuI2dGVlfy
----------------------------------------------------------------------------------------------------
پدرام شهلایی شاعر و خواننده ترانه «وره مەیدان/بیا به میدان» در یک ویدیو اعلام کرد که آهنگ خود را برای انقلاب #ژینا سروده و ظاهرا افراد سلطنت طلب (ایرانیان کرد میهن پرست ) در حال سواستفاده از این آهنگ برای منافع حزبی خود هستند . 
#زن_زندگى_آزادى https://t.co/JjFUWDmlHg
--------

In [25]:
show_tweets(user_quoted_tweets, 'user_quoted_tweets')


----------------------------------------------------------------------------------------------------
Number of user_quoted_tweets: 25 

----------------------------------------------------------------------------------------------------
در حاشیە سمیناری در رابطه با حقوق بشر در ایران در  مجمع جایزه صلح نوبل 
 کاک #عبداللە_مهتدی دبیر کل حزب #کومله کوردستان ایران و مسعود قرهخانی رئیس پارلمان نروژ سیاستمدار ایرانیتبار و عضو حزب کارگر نروژ
----------------------------------------------------------------------------------------------------
شکایت دو سازمان حقوق بشری از رئیس جمهور اسلامی ایران

ابراهیم #رئیسی رئیس جمهور اسلامی ایران قرار است روز چهارشنبه "۲۲ آذر ماە ۱٤٠٢" برای سخنرانی در نشستی به ژنو سفر کند. 
تلاش زیادی برای سازماندهی تظاهرات در مقابل محل برگزاری او انجام شده است اما تاکنون تلاش ها بی نتیجه بوده است.
در همین حال امروز به نمایندگی از سازمان حقوق بشر زاگرس و سازمان حقوق بشری #هانا از رئیسی به دلیل دست داشتن در جنایت علیه بشریت شکایت کردیم.
این شکایت بر اساس مواد 264a، 264b، 264

In [26]:
show_tweets(user_retweeted_tweets, 'user_retweeted_tweets')


----------------------------------------------------------------------------------------------------
Number of user_retweeted_tweets: 40 

----------------------------------------------------------------------------------------------------
در حاشیە سمیناری در رابطه با حقوق بشر در ایران در  مجمع جایزه صلح نوبل 
 کاک #عبداللە_مهتدی دبیر کل حزب #کومله کوردستان ایران و مسعود قرهخانی رئیس پارلمان نروژ سیاستمدار ایرانیتبار و عضو حزب کارگر نروژ https://t.co/vGxA19NjZH
----------------------------------------------------------------------------------------------------
اگه همه مدعوین رو تقی رحمانی دعوت نکرده نشون میده اروپا به شعور فارس تمامیتخواه شیعه شک داره! حالا هی برید زر بزنید ما به فدرال اعتماد نداریم! قبر جد و آبادتون که ندارید. فک کردید این دفعه خامنهای رو ور میدارن میر چسین رو میذارن یا علی افشاری رو؟ 😂😂😂
 https://t.co/zJgnK7yl23
----------------------------------------------------------------------------------------------------
دو پرده : حامی مردم ایران علیه تبلیغات رژیم جمهوری اسلام

In [27]:
show_tweets(user_replied_tweets, 'user_replied_tweets')


----------------------------------------------------------------------------------------------------
Number of user_replied_tweets: 46 

----------------------------------------------------------------------------------------------------
@ordinaryman1980 هنوز نفهمیدی #کردستان_گورستان_فاشیستان است مردک فاشیست ؟
----------------------------------------------------------------------------------------------------
@givHeisenberg کجا میفروشن ؟
----------------------------------------------------------------------------------------------------
@MansouriSirwan @MOMO_MIGAA منمبا شما موافقم آقای منصوری
----------------------------------------------------------------------------------------------------
@andfogle @hasrat1 یک امریکایی فهمید ولی سلطنت طلب هرگز نفهمید
----------------------------------------------------------------------------------------------------
@KeyOne367 ذهن ئابرا گیان 😭چرا زبان پارسی را پاس نمیداری ؟ از طرف حزب ایرانیان کرد میهن پرست🤣
-----------------------------------------

In [28]:
show_tweets(explore_tweets, 'explore_tweets')


----------------------------------------------------------------------------------------------------
Number of explore_tweets: 88 

----------------------------------------------------------------------------------------------------
صحبت های امروز اقای #تقی_رحمانی در کلاب هواس پیرو تمامی حواشی نسبت به مراسم نوبل.. ( ضمن اینکه میدانم درد ج ا و پهلوی و پهلوی گراها چیز دیگریست، این رو توییت کردم خطاب به بسیار از غیرپهلوی گراها که شخصا انها را مسبب اصلی جو موجود نوبل/نرگس ستیزی میدانم.. ) 

#نرگس_محمدى 
#نوبل https://t.co/nYrxb4r2b3
----------------------------------------------------------------------------------------------------
ستوری امروز سونیا شریفی در اینستاگرام، که نشون میده بعد از یکسال چگونه آپگریت شده و به جای اینکه مثل چارتا جاش بی هویت، روی خودش قیمت بذاره، به کورد بودنش افتخار میکنه، و چشم در چشم قاتلان، از آخرش حرف میزنه:
"مسیرش سخته، ولی آخرش قشنگه"
#سونیا_شریفی
#ژن_ژیان_ئازادی https://t.co/5vLcvOsfPB
-------------------------------------------------------------------------

In [29]:
end_tweet_recommending_t = datetime.datetime.now()
print(end_tweet_recommending_t)
print("The total time spent finding and displaying the user's favorite tweets:",
      end_tweet_recommending_t-start_calculating_matrix)


2024-02-02 04:39:12.729951
The total time spent finding and displaying the user's favorite tweets: 0:02:27.461180


# User Recommender 

In [30]:
start_user_recommending_t = datetime.datetime.now()
print(start_user_recommending_t)


2024-02-02 04:39:12.735299


In [31]:
def extract_users(x):
    tweet_info = json.loads(x)

    uids = [tweet_info["user"].get('id')]

    origin_uid = [tweet_info.get('in_reply_to_user_id_str', None),
                  tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
                  tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
                 ]

    origin_uid = [i for i in origin_uid if i is not None]

    if origin_uid:
        uids.append(origin_uid[0])
    
    return uids

In [32]:
recommended_users = tweets_rdd.filter(lambda x: json.loads(x)['id'] in user_favorites[0][1].keys() or
                                               json.loads(x)['user']['id'] == q_uid)\
                              .flatMap(extract_users).collect()
                            


In [33]:
counts = Counter(recommended_users)
mcru = counts.most_common(20)  # most_common_recommended_users
mcru_id, repeats = zip(*mcru[1:])
mcru[1:]

[('1201194426367979521', 16),
 ('1001931386293510145', 13),
 ('867126568451280896', 9),
 ('1521456975233622016', 8),
 ('1505185434799951872', 7),
 ('1279163323246878720', 7),
 ('1441247691514597382', 5),
 ('1455566016013012998', 5),
 ('1043181897088729088', 5),
 ('334161563', 5),
 ('1246515349803106305', 4),
 ('1530212779797450752', 4),
 ('1647536982086524928', 4),
 ('18409325', 4),
 ('1265342325485711363', 4),
 ('1704397331431362560', 4),
 ('1300686064646942727', 3),
 ('1445120935141466112', 3),
 ('542347917', 3)]

### Show Tweets of recommended Users

In [34]:
start_extract_recommended_users_tweets_t = datetime.datetime.now()
print(start_extract_recommended_users_tweets_t)


2024-02-02 04:39:45.095288


In [35]:
rec_users_tweets = tweets_rdd.filter(lambda x: json.loads(x)['user']['id'] in mcru_id)\
                            .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                            .collect()



In [36]:
end_extract_recommended_users_tweets_t = datetime.datetime.now()
print(end_extract_recommended_users_tweets_t)



2024-02-02 04:40:08.094141


#### Tweets of Query User

In [37]:
favorites_tweets = favorites_tweets2.copy()
print('','#'*100,'\n', 'Tweets of Query User\n', '#'*100,'\n')
show_tweets(extract_user_origin_tweets(favorites_tweets, q_uid, rem=False), 'user_origin_tweets')
show_tweets(extract_user_quoted_tweets(favorites_tweets, q_uid, rem=False), 'user_quoted_tweets')
show_tweets(extract_user_retweeted_tweets(favorites_tweets, q_uid, rem=False), 'user_retweeted_tweets')
show_tweets(extract_user_replied_tweets(favorites_tweets, q_uid, rem=False), 'user_replied_tweets')

 #################################################################################################### 
 Tweets of Query User
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 4 

----------------------------------------------------------------------------------------------------
ای مگس عرصه سیمرغ نه جولانگه توست
عرض خود میبری و زحمت ما میداری
تو به تقصیر خود افتادی از این در محروم
از که مینالی و فریاد چرا میداری

#رضا_عقده_ایی 
#حزب_کارتنی https://t.co/MpmdadysLc
----------------------------------------------------------------------------------------------------
نمایشگاه «آنچه باقی ماند»

 #فواد_مصطفی_سلطانی 
#اعدام_های_فرودگاه_سنندج https://t.co/MuI2dGVlfy
----------------------------------------------------------------------------------------------------
پدرام شهلایی شاعر و خواننده ترانه «وره مەیدان/بی

#### Tweets of Recommended User

In [38]:
for (recommended_uid, rep) in mcru[1:]:
    print('','#'*100,'\n', f'User ID: {recommended_uid}, repetitions: {rep}\n', '#'*100,'\n')
    show_tweets(extract_user_origin_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_origin_tweets')
    show_tweets(extract_user_quoted_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_quoted_tweets')
    show_tweets(extract_user_retweeted_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_retweeted_tweets')
    show_tweets(extract_user_replied_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_replied_tweets')



 #################################################################################################### 
 User ID: 1201194426367979521, repetitions: 16
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 15 

----------------------------------------------------------------------------------------------------
اهدای جایزه ساخاروف به صالح نیکبخت وکیل خانواده #ژینا_امینی

#زن_زندگى_آزادى 
#مهسا_امينی https://t.co/Ij7ZmJY8N4
----------------------------------------------------------------------------------------------------
کاردو میری؛
با نگاهی گذرا به عقده گشایی مندرج در روزنامه حکومتی صبح نو، به موتیفهای؛ کومله، #عبدالله_مهتدی، زن زندگی آزادی، تجزیه طلب، آشوب خیابانی، ایران و غیره برمیخوریم. 
بسامد چنین واژههایی حاکی از عمق نگرانی و در عین حال اذعان به اهمیت حضور #کومله در مراسم اعطای جایزه صلح نوبل است.
در عین

In [39]:
end_notebook_t = datetime.datetime.now()
print(end_notebook_t,'\n')

2024-02-02 04:40:08.171025 



In [40]:
print("The total time spent for showing a sample of tweets and count tweets:      ", str(start_calculating_matrix - start_notebook_t).split('.')[0])
print("The total time spent for calculating Primary Rating Matrix:                ", str(start_extracting_tweets_id_t - start_calculating_matrix).split('.')[0])
print("The total time spent for extracting Recommended Tweets' ID:                ", str(start_extract_and_show_tweets_t - start_extracting_tweets_id_t).split('.')[0])
print("The total time spent for extract and show Tweets:                          ", str(start_user_recommending_t - start_extract_and_show_tweets_t).split('.')[0])
print("The total time spent for extracting Recommended Ussers:                    ", str(start_extract_recommended_users_tweets_t - start_user_recommending_t).split('.')[0])
print("The total time spent for extract and show Tweets' of Recommended Users:    ", str(end_extract_recommended_users_tweets_t - start_extract_recommended_users_tweets_t).split('.')[0])
print('_'*90)
print("The total time spent for this script:                                      ", str(end_notebook_t-start_notebook_t).split('.')[0])


The total time spent for showing a sample of tweets and count tweets:       0:00:03
The total time spent for calculating Primary Rating Matrix:                 0:00:36
The total time spent for extracting Recommended Tweets' ID:                 0:01:18
The total time spent for extract and show Tweets:                           0:00:31
The total time spent for extracting Recommended Ussers:                     0:00:32
The total time spent for extract and show Tweets' of Recommended Users:     0:00:22
__________________________________________________________________________________________
The total time spent for this script:                                       0:03:26
